## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x,a):
    return 2.*x/(x + float(a)) - 1.

def f_invtransform(s,a):
    return float(a)*(1. + s)/(1. - s)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=img_size
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict
    

In [6]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

In [7]:
img_size=64
kappa=10
# img_size=128
# kappa=50

In [8]:
### Read validation data
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins,kappa)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=100
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data   
for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:,:]
    
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


In [9]:
df_runs=pd.DataFrame([])

In [10]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.8
for count,run in enumerate(lst):
    num=100
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(run)
    print(fname)    
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:,:]
    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num

    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


### Add pytorch runs

In [13]:
## Pytorch code
### Load images

# Run 1
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'
fldr='20210929_32023_cgan_bs32_nodes8_lr0.0001-fixed_fm10_spec0.0_kappa10_goodrun/'
lst=[42770,43060,48560,48570]


In [14]:
parent_dir+=fldr

# Prediction images 

for param_label in sigma_lst:
    prefix='pred_%s_'%(param_label)    
    for count,run in enumerate(lst):
        step=run
        key=prefix+'{0}'.format(step)
        print(key)
        fname=glob.glob(parent_dir+'images/'+'inference_*label-{0}_*_step-{1}.npy'.format(param_label,step))[0]
        dict1={'label':key,'fname':fname}
        images=np.load(dict1['fname'])     ### No transform needed here

        images=images[:,0,:,:,:]
        print(images.shape)
        print(np.max(images),np.min(images))

        dict1['num_imgs']=images.shape[0]
        ### Compute spectrum and histograms
        dict_sample=f_compute_hist_spect(images,bins)
        ### Compute chi squares
        dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

        dict1.update(dict_sample)
        dict1.update(dict_chisqrs)
        del(images)
        df_runs=df_runs.append(dict1,ignore_index=True)

pred_0.5_42770
(128, 64, 64, 64)
0.9929335 -0.9991739
pred_0.5_43060
(128, 64, 64, 64)
0.9875465 -0.9994661
pred_0.5_48560
(128, 64, 64, 64)
0.99045753 -0.9994027
pred_0.5_48570
(128, 64, 64, 64)
0.98919135 -0.9990813
pred_0.65_42770
(128, 64, 64, 64)
0.99741757 -0.9996155
pred_0.65_43060
(128, 64, 64, 64)
0.9933707 -0.99969566
pred_0.65_48560
(128, 64, 64, 64)
0.9932272 -0.99981695
pred_0.65_48570
(128, 64, 64, 64)
0.9926307 -0.99963796
pred_0.8_42770
(128, 64, 64, 64)
0.9984816 -0.999813
pred_0.8_43060
(128, 64, 64, 64)
0.9959239 -0.9998564
pred_0.8_48560
(128, 64, 64, 64)
0.99675715 -0.9997407
pred_0.8_48570
(128, 64, 64, 64)
0.9942419 -0.99981093
pred_1.1_42770
(128, 64, 64, 64)
0.99820477 -0.9999448
pred_1.1_43060
(128, 64, 64, 64)
0.9991416 -0.9999186
pred_1.1_48560
(128, 64, 64, 64)
0.996616 -0.9999248
pred_1.1_48570
(128, 64, 64, 64)
0.99604845 -0.99987924


In [15]:
# Train images 
for param_label in sigma_lst:
    prefix='train_%s_'%(param_label)
    lst=[42770,43060,48560,48570]
    for count,run in enumerate(lst):
        step=run
        key=prefix+'{0}'.format(step)
        print(key)
        try: 
            fname=glob.glob(parent_dir+'images/'+'gen_img_label-{0}_*_step-{1}.npy'.format(param_label,step))[0]
            dict1={'label':key,'fname':fname}
            images=np.load(dict1['fname'])     ### No transform needed here

            images=images[:,0,:,:,:]
            print(images.shape)
            print(np.max(images),np.min(images))

            dict1['num_imgs']=images.shape[0]
            ### Compute spectrum and histograms
            dict_sample=f_compute_hist_spect(images,bins)
            ### Compute chi squares
            dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

            dict1.update(dict_sample)
            dict1.update(dict_chisqrs)
            del(images)
            df_runs=df_runs.append(dict1,ignore_index=True)
        except: 
            pass

train_0.5_42770
(16, 64, 64, 64)
0.9821556 -0.9980623
train_0.5_43060
(16, 64, 64, 64)
0.9784355 -0.99858004
train_0.5_48560
(16, 64, 64, 64)
0.9937606 -0.9989644
train_0.5_48570
(16, 64, 64, 64)
0.99090713 -0.9988727
train_0.65_42770
train_0.65_43060
train_0.65_48560
train_0.65_48570
train_0.8_42770
(16, 64, 64, 64)
0.9940408 -0.99953514
train_0.8_43060
(16, 64, 64, 64)
0.99281514 -0.9994883
train_0.8_48560
(16, 64, 64, 64)
0.9909371 -0.9997091
train_0.8_48570
(16, 64, 64, 64)
0.9884784 -0.9997071
train_1.1_42770
(16, 64, 64, 64)
0.99676305 -0.9998732
train_1.1_43060
(16, 64, 64, 64)
0.99115133 -0.999897
train_1.1_48560
(16, 64, 64, 64)
0.9958787 -0.99992555
train_1.1_48570
(16, 64, 64, 64)
0.9921326 -0.9999447


In [16]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'spec_sdev', 'spec_val'],
      dtype='object')

In [17]:
col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [18]:
df_runs

,label,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.538356,0.423050,1.674713,49.823161,0.087058,0.028248,0.464351,0.610389,247.628749,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.008008086373939193, 0.0013947711012674978, ...","[0.5543578548431399, 1.3215428638458249, 0.160...","[1882799.6425010688, 643482.7311988472, 289724...","[6562187.582494507, 3137712.6836604346, 169973...",100.0,
1,ref_0.65,0.123235,0.089259,0.311717,47.083935,0.022825,0.011150,0.098855,0.913559,48.915385,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.012726906582800263, 0.002375973956342442, 0...","[0.902475628852844, 1.2083192911148068, 0.1468...","[2309170.8587837527, 822010.1436098838, 378801...","[8220451.7131622825, 3776521.3506506598, 19760...",100.0,
2,ref_0.8,0.005822,0.004351,0.062368,41.332151,0.000837,0.000634,0.003234,1.152376,6.935490,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.015548622861749229, 0.0030507710783071176, ...","[1.254987602233887, 1.099098100376129, 0.13191...","[2950050.666157404, 1097121.6856185866, 554871...","[9500922.004443746, 4356143.589759363, 2236216...",100.0,
3,ref_1.1,0.430938,0.379962,0.843879,48.713291,0.011431,0.039545,0.443918,1.581118,129.158764,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.02291234050517898, 0.005076758705673229, 0....","[1.8381116201137886, 0.9205307224134657, 0.111...","[3885506.4092630157, 1498283.4762213933, 73990...","[11170633.921163272, 4847186.2042914275, 25157...",100.0,
4,pred_0.5_42770,5.044223,4.982987,84.390439,50.479630,0.058516,0.002721,2.709588,0.986699,5311.631952,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.014031209620412391, 0.003960674190387762, 0...","[2.1309741181467463, 0.9011791630983259, 0.041...","[898576.2359491755, 277403.9412935678, 148945....","[2728088.939402403, 1230630.9144439413, 667406...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pred_0.5_43060,5.969822,5.906839,84.401369,50.673315,0.060131,0.002852,3.237955,0.962202,5006.279343,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.013927311837850004, 0.00392415722534012, 0....","[2.277523239701988, 0.8535280247032645, 0.0410...","[809002.4433310312, 278879.07278828457, 146585...","[2613957.9594374774, 1264564.7548724422, 68163...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pred_0.5_48560,6.470662,6.406109,97.561845,50.485573,0.061867,0.002685,3.519104,0.991659,7456.751750,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.014569416550025471, 0.004208059341887062, 0...","[2.350814711123343, 0.8313920749514179, 0.0386...","[907761.5219996297, 293808.2578216278, 131899....","[2651331.5605080444, 1254121.5769859378, 66007...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pred_0.5_48570,5.742108,5.676430,87.221361,50.647192,0.062690,0.002988,3.102785,0.931371,6243.957580,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.013472578099600195, 0.0038013643329490683, ...","[2.241373322904112, 0.8661272366344931, 0.0403...","[870828.3029079614, 293537.53015226277, 136154...","[2425908.2487639533, 1166408.7969493682, 64340...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pred_0.65_42770,3.802265,3.707156,71.357229,51.016075,0.089340,0.005769,3.141843,0.824717,4476.154566,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.015492229221951316, 0.004442066111296134, 0...","[2.5883430315717155, 0.7467240161842822, 0.044...","[1197371.8384681668, 340615.5645364227, 203794...","[3321570.1839062017, 1501583.4479499273, 83604...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pred_0.65_43060,4.289557,4.185170,80.914930,51.171448,0.095460,0.008928,3.564785,0.818179,4455.613584,"[-1.0050125313283207, -0.5975422427035331, -0....","[0.015736077921807333, 0.004502137536884083, 0...","[2.6988881530164264, 0.7125096251774761, 0.042...","[1138313.1746907837, 367097.97472380847, 18775...",

### Compare chi-squares

In [19]:
chi_keys=['label','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,ref_0.5,0.538356,1.674713,100.0,0.423050,0.087058,0.028248,0.464351,0.610389,247.628749,49.823161
1,ref_0.65,0.123235,0.311717,100.0,0.089259,0.022825,0.011150,0.098855,0.913559,48.915385,47.083935
2,ref_0.8,0.005822,0.062368,100.0,0.004351,0.000837,0.000634,0.003234,1.152376,6.935490,41.332151
3,ref_1.1,0.430938,0.843879,100.0,0.379962,0.011431,0.039545,0.443918,1.581118,129.158764,48.713291
4,pred_0.5_42770,5.044223,84.390439,128.0,4.982987,0.058516,0.002721,2.709588,0.986699,5311.631952,50.479630
5,pred_0.5_43060,5.969822,84.401369,128.0,5.906839,0.060131,0.002852,3.237955,0.962202,5006.279343,50.673315
6,pred_0.5_48560,6.470662,97.561845,128.0,6.406109,0.061867,0.002685,3.519104,0.991659,7456.751750,50.485573
7,pred_0.5_48570,5.742108,87.221361,128.0,5.676430,0.062690,0.002988,3.102785,0.931371,6243.957580,50.647192
8,pred_0.65_42770,3.802265,71.357229,128.0,3.707156,0.089340,0.005769,3.141843,0.824717,4476.154566,51.016075
9,pred_0.65_43060,4.289557,80.914930,128.0,4.185170,0.095460,0.008928,3.564785,0.818179,4455.613584,51.171448


## Plot

In [20]:
def f_plot_hist_spec(df,dict_bkg,plot_type,img_size):

    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1,kappa)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
#             yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            yerr2=row.spec_sdev

            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)
    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.legend()
            plt.title("relative spectrum")
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))

            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x,kappa)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
#         y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']

        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        plt.xlim(0,img_size/2)


    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.title("relative spectrum")
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)    
    
    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.5, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.show()
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [21]:
# Define the reference 
param_label=0.8
dict_val=bkgnd_dict[str(param_label)]

In [22]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type,img_size)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                img_size=fixed(img_size),
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('ref_0.5', 'ref_0.65', 'ref_0.8', 're…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [23]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,ref_0.5,0.538356,1.674713,100.0,0.464351,247.628749
1,ref_0.65,0.123235,0.311717,100.0,0.098855,48.915385
2,ref_0.8,0.005822,0.062368,100.0,0.003234,6.935490
3,ref_1.1,0.430938,0.843879,100.0,0.443918,129.158764
4,pred_0.5_42770,5.044223,84.390439,128.0,2.709588,5311.631952
5,pred_0.5_43060,5.969822,84.401369,128.0,3.237955,5006.279343
6,pred_0.5_48560,6.470662,97.561845,128.0,3.519104,7456.751750
7,pred_0.5_48570,5.742108,87.221361,128.0,3.102785,6243.957580
8,pred_0.65_42770,3.802265,71.357229,128.0,3.141843,4476.154566
9,pred_0.65_43060,4.289557,80.914930,128.0,3.564785,4455.613584


In [24]:
dict_val

{'hist_val': array([1.20020737e+00, 1.10988262e+00, 1.39227382e-01, 7.06068652e-02,
        4.88826247e-02, 3.82954909e-02, 3.21166094e-02, 2.81631226e-02,
        2.51859850e-02, 2.28181040e-02, 2.13548309e-02, 2.01127582e-02,
        1.89135673e-02, 1.81787799e-02, 1.71590529e-02, 1.59791628e-02,
        1.58064466e-02, 1.54330112e-02, 1.45022155e-02, 1.43665192e-02,
        1.36436375e-02, 1.29298392e-02, 1.18451062e-02, 1.08310954e-02,
        9.97413683e-03, 9.02524853e-03, 8.54190650e-03, 8.29284825e-03,
        7.65761340e-03, 6.78956094e-03, 6.73549612e-03, 6.08473706e-03,
        5.27260679e-03, 4.21850667e-03, 3.85614996e-03, 2.90344149e-03,
        2.57150628e-03, 2.13812636e-03, 2.13031558e-03, 1.64897232e-03,
        1.34025418e-03, 1.54309252e-03, 9.86290483e-04, 9.00743551e-04,
        2.66658025e-04]),
 'hist_err': array([1.31839102e-02, 2.68076225e-03, 1.82248261e-03, 1.19061907e-03,
        9.37100175e-04, 8.05554742e-04, 7.18839733e-04, 6.61824144e-04,
        6.0898

### Test 

In [ ]:
df_runs.fname.values

In [ ]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210112_191125_cgan_predict_1.1_m2/images/inference_spec_label-1.1_epoch-11_step-25999.npy'
# a1=np.load(fname)
# print(a1.shape)

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy'
val_arr=np.load(fname)[:2000][:,0,:,:]
print(val_arr.shape)

In [ ]:
f_pixel_intensity(f_invtransform(a1,kappa),normalize=True)
plt.xscale('symlog',linthreshx=50)


# img_lst=[a1]
# f_compare_pixel_intensity(img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


In [ ]:
val_arr=val_arr[:,0,:,:]

In [ ]:
img_lst=[a1,val_arr]
f_compare_spectrum(img_lst=img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


### Test histogram

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy'
print(fname)
samples=np.load(fname,mmap_mode='r')[-1000:][:,0,:,:]
print(samples.shape)